<a href="https://colab.research.google.com/github/CatIsOutOfTheBag/PetSQL/blob/main/SQL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GROUP BY

Подготовка датафрейма и базы

In [ ]:
# Подключаем библиотеки
import pandas as pd
import numpy as np
import sqlite3

https://drive.google.com/file/d/1Gj0qVU0Acv-HqRjTIEwvkO4TqJYijehR/view?usp=sharing

In [ ]:
# Заливаем данные
!gdown --id 1Gj0qVU0Acv-HqRjTIEwvkO4TqJYijehR

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Gj0qVU0Acv-HqRjTIEwvkO4TqJYijehR
To: /content/german_credit_augmented.csv
100% 69.6k/69.6k [00:00<00:00, 72.1MB/s]


In [ ]:
# Сохраняем данные в датафрейм
df = pd.read_csv('/content/german_credit_augmented.csv')

In [ ]:
# Контролируем тип ДАТА, так как он заливается просто как текст
df['contract_dt'] = pd.to_datetime(df['contract_dt'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
# Создаем файл с базой данных на гугл-диске
con = sqlite3.connect('/content/drive/My Drive/german_credit.db')

In [ ]:
# Сохраняем таюлицу в базу
df.to_sql('german_credit', con, index=False, if_exists='replace')


Начало работы

In [ ]:
def select(sql):
  return pd.read_sql(sql, con)

Задача:

найти средний размер кредита по полу и количество человек в каждой категории

In [ ]:
sql = '''select t.sex,
count(*) as cnt,
avg(t.credit_amount) as credit_amount_avg
from german_credit t
group by t.sex
'''
select(sql)

,sex,cnt,credit_amount_avg
0,female,310,2877.774194
1,male,690,3448.040580


In [ ]:
# Аналог в пандас:
df.groupby('sex')['credit_amount'].agg(['count', 'mean'])

,count,mean
sex,,
female,310,2877.774194
male,690,3448.040580


In [ ]:
# Уникальные значения в поле hosing
sql = '''select distinct t.housing from german_credit t'''
select(sql)

,housing
0,own
1,rent
2,free


In [ ]:
# Колчисество уникальных значений в поле hosing
sql = '''select count(distinct t.housing), count(t.housing) from german_credit t'''
select(sql)

,count(distinct t.housing),count(t.housing)
0,3,1000


In [ ]:
# сводная таблица по housing
sql = '''select t.housing,
count(*) as cnt,
avg(t.credit_amount) as credit_amount_avg
from german_credit t
group by t.housing
'''
select(sql)

,housing,cnt,credit_amount_avg
0,free,108,4906.212963
1,own,713,3060.939691
2,rent,179,3122.553073


#Пропуски

In [ ]:
sql = '''select * from german_credit t'''
select(sql)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,None,None,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,None,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,None,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,None,776,12,radio/TV,0,2007-07-21 13:22:14,358
...,...,...,...,...,...,...,...,...,...,...,...,...
995,65,male,2,free,little,little,2600,18,radio/TV,1,2007-12-16 20:17:19,624
996,30,male,3,own,little,moderate,4455,36,business,1,2007-07-12 14:08:58,181
997,33,male,2,own,little,moderate,6403,24,radio/TV,0,2008-04-08 03:24:26,730
998,29,female,2,own,None,None,5003,21,car,1,2007-11-29 15:51:45,557


In [ ]:
sql = '''select t.checking_account from german_credit t'''
select(sql)

,checking_account
0,None
1,little
2,None
3,None
4,None
...,...
995,little
996,moderate
997,moderate
998,None


In [ ]:
sql = '''select count(t.checking_account) from german_credit t''' # видим, что есть пропуски
select(sql)

,count(t.checking_account)
0,606


In [ ]:
# Сделаем сводник
sql = '''select t.checking_account,
count(*) as cnt,
avg(t.credit_amount) as credit_amount_avg
from german_credit t
group by t.checking_account
'''
select(sql)

,checking_account,cnt,credit_amount_avg
0,None,394,3133.101523
1,little,274,3175.218978
2,moderate,269,3827.561338
3,rich,63,2177.650794


Сводные таблицы в SQL учитывают пропуски

In [ ]:
# Сделаем поле с флагами, указывающими на пропуски

sql = '''select t.checking_account,
case when t.checking_account is null then 1 else 0 end as is_null
from german_credit t
'''
select(sql)

,checking_account,is_null
0,None,1
1,little,0
2,None,1
3,None,1
4,None,1
...,...,...
995,little,0
996,moderate,0
997,moderate,0
998,None,1


In [ ]:
# просуммировав это поле, получим количество пропусков
select(sql)['is_null'].sum()

394

In [ ]:
# или вот так
sql = '''select
sum(case when t.checking_account is null then 1 else 0 end) as is_null
from german_credit t
'''
select(sql)

,is_null
0,394


Заполнение пропусков

In [ ]:
sql = '''select
t.checking_account
from german_credit t
'''
select(sql)

,checking_account
0,None
1,little
2,None
3,None
4,None
...,...
995,little
996,moderate
997,moderate
998,None


In [ ]:
sql = '''select
t.checking_account,
coalesce(t.checking_account, 'no_info')
from german_credit t
'''
select(sql)

,checking_account,"coalesce(t.checking_account, 'no_info')"
0,None,no_info
1,little,little
2,None,no_info
3,None,no_info
4,None,no_info
...,...,...
995,little,little
996,moderate,moderate
997,moderate,moderate
998,None,no_info


In [ ]:
sql = '''select
coalesce(t.checking_account, 'no_info')
from german_credit t
'''
select(sql)

,"coalesce(t.checking_account, 'no_info')"
0,no_info
1,little
2,no_info
3,no_info
4,no_info
...,...
995,little
996,moderate
997,moderate
998,no_info


In [ ]:
df.head()

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,NaN,NaN,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,NaN,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,NaN,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,NaN,776,12,radio/TV,0,2007-07-21 13:22:14,358


In [ ]:
# а вот так coalesce найдет первый НЕ нан в строке из перечисленных столбцов
sql = '''select
t.checking_account, t.saving_accounts,
coalesce(t.checking_account, t.saving_accounts, 'no_info')
from german_credit t
'''
select(sql)

,checking_account,saving_accounts,"coalesce(t.checking_account, t.saving_accounts, 'no_info')"
0,None,None,no_info
1,little,little,little
2,None,quite rich,quite rich
3,None,little,little
4,None,little,little
...,...,...,...
995,little,little,little
996,moderate,little,moderate
997,moderate,little,moderate
998,None,None,no_info


#Дубликаты

In [ ]:
# на игрушечных данных
t = pd.DataFrame({'id': [1,1,2],
                  'name': ['a','a','b']})

In [ ]:
t

,id,name
0,1,a
1,1,a
2,2,b


In [ ]:
t.to_sql('duples', con, index=False, if_exists='replace')

In [ ]:
sql = '''select * from duples t'''
select(sql)

,id,name
0,1,a
1,1,a
2,2,b


In [ ]:
# пропуски можно увидеть, сгруппировав по всем столбцам
sql = '''select t.id, t.name, count(*) as cnt from duples t
group by t.id, t.name
'''
select(sql)

,id,name,cnt
0,1,a,2
1,2,b,1


In [ ]:
# Увидеть только дубликаты
sql = '''select t.id, t.name, count(*) as cnt from duples t
group by t.id, t.name
'''
select(sql)

Чтобы выбрать строки по условию с функцией агрегации, нужно использовать HAVING

# HAVING

In [ ]:
sql = '''select t.id, t.name, count(*) as cnt from duples t
group by t.id, t.name
having count(*) > 1
'''
select(sql)

,id,name,cnt
0,1,a,2


Если дуплятся только id

In [ ]:
# на игрушечных данных
t = pd.DataFrame({'id': [1,1,2,2,3],
                  'name': ['a','b','c','d','e']})

In [ ]:
t.to_sql('duples', con, index=False, if_exists='replace')

In [ ]:
sql = '''select * from duples t'''
select(sql)

,id,name
0,1,a
1,1,b
2,2,c
3,2,d
4,3,e


In [ ]:
# Увидеть дубликаты id
sql = '''select t.id, count(*) as cnt from duples t
group by t.id
having count(1) > 1
'''
select(sql)

,id,cnt
0,1,2
1,2,2


Подзапрос

In [ ]:
# это id, которые повторяются
sql = '''select t.id from duples t
group by t.id
having count(1) > 1
'''
select(sql)

,id
0,1
1,2


In [ ]:
# посмотреть все столбцы для id, которые повторяются
sql = '''select * from duples t
where t.id in (select t.id from duples t
group by t.id
having count(1) > 1)'''
# сначала выполнится то, что в скобках
select(sql)


,id,name
0,1,a
1,1,b
2,2,c
3,2,d


# Агрегирование = группировка

Сделаем отчет, в котором посчитано количество кредитов на каждый месяц

In [ ]:
sql = '''select * from german_credit t'''
select(sql).head()

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,None,None,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,None,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,None,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,None,776,12,radio/TV,0,2007-07-21 13:22:14,358


In [ ]:
# сначала нужно взять месяц = первый день месяца
sql = '''select date(t.contract_dt, 'start of month') as month,
count(*) as credit_cnt
from german_credit t
group by date(t.contract_dt, 'start of month')
order by date(t.contract_dt, 'start of month')
'''
select(sql)

,month,credit_cnt
0,2007-05-01,81
1,2007-06-01,74
2,2007-07-01,71
3,2007-08-01,57
4,2007-09-01,58
5,2007-10-01,70
6,2007-11-01,87
7,2007-12-01,77
8,2008-01-01,93
9,2008-02-01,55


In [ ]:
# можно добавить функции агрегации для величин кредитов:
# - количество
# - сумма
# - среднее
sql = '''select date(t.contract_dt, 'start of month') as month,
count(*) as credit_cnt,
sum(t.credit_amount) as credit_amount_sum,
avg(t.credit_amount) as credit_amount_avg
from german_credit t
group by date(t.contract_dt, 'start of month')
order by date(t.contract_dt, 'start of month')
'''
select(sql)

,month,credit_cnt,credit_amount_sum,credit_amount_avg
0,2007-05-01,81,207663,2563.740741
1,2007-06-01,74,239594,3237.756757
2,2007-07-01,71,224333,3159.619718
3,2007-08-01,57,178569,3132.789474
4,2007-09-01,58,186909,3222.568966
5,2007-10-01,70,188534,2693.342857
6,2007-11-01,87,300504,3454.068966
7,2007-12-01,77,273973,3558.090909
8,2008-01-01,93,288080,3097.634409
9,2008-02-01,55,211128,3838.690909


Рассмотрим вариант, когда id клиентов не уникальны - когда один человек берет несколько кредитов

Смоделируем эту ситуацию

In [ ]:
df['client_id_2'] = pd.Series(range(600)).sample(1000, replace=True).values

In [ ]:
df.to_sql('german_credit_not_unique', con, index=False, if_exists='replace')

In [ ]:
sql = '''select date(t.contract_dt, 'start of month') as month,
count(*) as credit_cnt,
count(distinct t.client_id_2) as client_id_unique,
sum(t.credit_amount) as credit_amount_sum,
avg(t.credit_amount) as credit_amount_avg
from german_credit_not_unique t
group by date(t.contract_dt, 'start of month')
order by date(t.contract_dt, 'start of month')
'''
select(sql)

,month,credit_cnt,client_id_unique,credit_amount_sum,credit_amount_avg
0,2007-05-01,81,78,207663,2563.740741
1,2007-06-01,74,69,239594,3237.756757
2,2007-07-01,71,67,224333,3159.619718
3,2007-08-01,57,53,178569,3132.789474
4,2007-09-01,58,56,186909,3222.568966
5,2007-10-01,70,66,188534,2693.342857
6,2007-11-01,87,80,300504,3454.068966
7,2007-12-01,77,75,273973,3558.090909
8,2008-01-01,93,85,288080,3097.634409
9,2008-02-01,55,53,211128,3838.690909


# Группировка непрерывных величин

разбивка на бакеты (бины, интервалы)

In [ ]:
# Количество уникальных значений поля credit_amount
sql = '''select count(distinct t.credit_amount) from german_credit t'''
select(sql)

,count(distinct t.credit_amount)
0,921


In [ ]:
sql = '''select t.credit_amount,

case when t.credit_amount < 1000 then '1. <1000' 
when t.credit_amount < 2000 then '2. 1000-2000' 
when t.credit_amount < 3000 then '3. 2000-3000' 
when t.credit_amount >= 3000 then '4. >= 3000' 


else 'other' end as credit_amount_bin
from german_credit t'''

select(sql)

,credit_amount,credit_amount_bin
0,3074,4. >= 3000
1,1344,2. 1000-2000
2,936,1. <1000
3,1393,2. 1000-2000
4,776,1. <1000
...,...,...
995,2600,3. 2000-3000
996,4455,4. >= 3000
997,6403,4. >= 3000
998,5003,4. >= 3000


In [ ]:
# посчитаем количество кедитов в каждой категории
sql = '''select

case when t.credit_amount < 1000 then '1. <1000' 
when t.credit_amount < 2000 then '2. 1000-2000' 
when t.credit_amount < 3000 then '3. 2000-3000' 
when t.credit_amount >= 3000 then '4. >= 3000' 
else 'other' end as credit_amount_bin,

count(*)
from german_credit t

group by case when t.credit_amount < 1000 then '1. <1000' 
when t.credit_amount < 2000 then '2. 1000-2000' 
when t.credit_amount < 3000 then '3. 2000-3000' 
when t.credit_amount >= 3000 then '4. >= 3000' 
else 'other' end

'''
select(sql)

,credit_amount_bin,count(*)
0,1. <1000,116
1,2. 1000-2000,316
2,3. 2000-3000,188
3,4. >= 3000,380


In [ ]:
# В sqlite3 можно использовать псевдонимы для меньшей громоздкости кода

sql = '''select

case when t.credit_amount < 1000 then '1. <1000' 
when t.credit_amount < 2000 then '2. 1000-2000' 
when t.credit_amount < 3000 then '3. 2000-3000' 
when t.credit_amount >= 3000 then '4. >= 3000' 
else 'other' end as credit_amount_bin,

count(*) as credit_cnt
from german_credit t

group by credit_amount_bin

'''
select(sql)

,credit_amount_bin,credit_cnt
0,1. <1000,116
1,2. 1000-2000,316
2,3. 2000-3000,188
3,4. >= 3000,380


Group by с несколькими столбцами

In [ ]:
# сначала сгруппируем по housing - увидим количество в каждой группе
sql = '''select t.housing, count(*) from german_credit t
group by t.housing
'''
select(sql)

,housing,count(*)
0,free,108
1,own,713
2,rent,179


In [ ]:
# теперь разобьем это по полу
# count(case when t.sex = 'female' then 1 else null end) - count посчитает только для female

sql = '''select t.housing,
count(case when t.sex = 'female' then 1 else null end) as female,
count(case when t.sex = 'male' then 1 else null end) as male,


count(*) as cnt
from german_credit t
group by t.housing
'''
select(sql)

,housing,female,male,cnt
0,free,19,89,108
1,own,196,517,713
2,rent,95,84,179


На питоне проще

Сделаем это для столбца purpose

In [ ]:
# сначала найдем все уникальные категории
sql = '''select distinct t.purpose from german_credit t'''

In [ ]:
# сохраним их в список
purpose = list(select(sql)['purpose'].values)

In [ ]:
# сгенерируем код для sql-запроса
for p in purpose:
  print(f"count(case when t.purpose = '{p}' then 1 else null end) as {p.lower().replace(' ','').replace('/', '')},")

count(case when t.purpose = 'radio/TV' then 1 else null end) as radiotv,
count(case when t.purpose = 'car' then 1 else null end) as car,
count(case when t.purpose = 'education' then 1 else null end) as education,
count(case when t.purpose = 'furniture/equipment' then 1 else null end) as furnitureequipment,
count(case when t.purpose = 'repairs' then 1 else null end) as repairs,
count(case when t.purpose = 'business' then 1 else null end) as business,
count(case when t.purpose = 'domestic appliances' then 1 else null end) as domesticappliances,
count(case when t.purpose = 'vacation/others' then 1 else null end) as vacationothers,


In [ ]:
sql = '''select t.housing,
count(case when t.purpose = 'radio/TV' then 1 else null end) as radiotv,
count(case when t.purpose = 'car' then 1 else null end) as car,
count(case when t.purpose = 'education' then 1 else null end) as education,
count(case when t.purpose = 'furniture/equipment' then 1 else null end) as furnitureequipment,
count(case when t.purpose = 'repairs' then 1 else null end) as repairs,
count(case when t.purpose = 'business' then 1 else null end) as business,
count(case when t.purpose = 'domestic appliances' then 1 else null end) as domesticappliances,
count(case when t.purpose = 'vacation/others' then 1 else null end) as vacationothers,

count(*) as cnt
from german_credit t
group by t.housing
'''
select(sql)

,housing,radiotv,car,education,furnitureequipment,repairs,business,domesticappliances,vacationothers,cnt
0,free,15,55,15,11,3,5,0,4,108
1,own,227,219,34,122,17,76,10,8,713
2,rent,38,63,10,48,2,16,2,0,179


# Агрегирование разрозненного текста

In [ ]:
purpose = ['машина','машина','машина','на машину','на покупку машины',
                             'автомобиль','на возвращение 2007', 
                             'на свадьбу','свадьба','свадьба','свадьба','для свадьбы',
                             'недвижимость','на покупку недвижимости']

In [ ]:
df = pd.DataFrame({'purpose': purpose})

In [ ]:
df

,purpose
0,машина
1,машина
2,машина
3,на машину
4,на покупку машины
5,автомобиль
6,на возвращение 2007
7,на свадьбу
8,свадьба
9,свадьба


In [ ]:
df.to_sql('purpose', con, index=False, if_exists='replace')

In [ ]:
sql = '''select * from purpose t'''
select(sql)

,purpose
0,машина
1,машина
2,машина
3,на машину
4,на покупку машины
5,автомобиль
6,на возвращение 2007
7,на свадьбу
8,свадьба
9,свадьба


In [ ]:
sql = '''select t.purpose, count(*) from purpose t
group by t.purpose
order by count(*) desc'''
select(sql)

,purpose,count(*)
0,машина,3
1,свадьба,3
2,автомобиль,1
3,для свадьбы,1
4,на возвращение 2007,1
5,на машину,1
6,на покупку машины,1
7,на покупку недвижимости,1
8,на свадьбу,1
9,недвижимость,1


In [ ]:
# начнем со свадьбы
cat = '''select t.purpose,
case when t.purpose like '%свадьб%' then 'свадьба'
else 'другое' end as purpose_cat

from purpose t
'''
select(cat)

,purpose,purpose_cat
0,машина,другое
1,машина,другое
2,машина,другое
3,на машину,другое
4,на покупку машины,другое
5,автомобиль,другое
6,на возвращение 2007,другое
7,на свадьбу,свадьба
8,свадьба,свадьба
9,свадьба,свадьба


In [ ]:
# посмотрим сколько попало в категорию свадьба
sql = f'''select
t.purpose_cat, 

count(*)

from ({cat}) t
group by t.purpose_cat
'''
select(sql)

,purpose_cat,count(*)
0,другое,9
1,свадьба,5


In [ ]:
# дальше смотрим, что попало в другое - действуем итерациями
# используем подзапрос, где мы создаем рассчетные столбец purpose_cut


sql = f'''select t.purpose, count(*)
from ({cat}) t
where t.purpose_cat = 'другое'

group by t.purpose
order by count(*) desc'''

select(sql)

,purpose,count(*)
0,машина,3
1,автомобиль,1
2,на возвращение 2007,1
3,на машину,1
4,на покупку машины,1
5,на покупку недвижимости,1
6,недвижимость,1


In [ ]:
# видим, что теперь на первом месте машина, добавляем его в like,
# повторяем создание категории

cat = '''select t.purpose,
case when t.purpose like '%свадьб%' then 'свадьба'
when (t.purpose like '%машин%' or t.purpose like '%авто%') then 'машина'
else 'другое' end as purpose_cat

from purpose t
'''
select(cat)


,purpose,purpose_cat
0,машина,машина
1,машина,машина
2,машина,машина
3,на машину,машина
4,на покупку машины,машина
5,автомобиль,машина
6,на возвращение 2007,другое
7,на свадьбу,свадьба
8,свадьба,свадьба
9,свадьба,свадьба


In [ ]:
# посмотрим сколько попало в категории свадьба, машина, другое
sql = f'''select
t.purpose_cat, 

count(*)

from ({cat}) t
group by t.purpose_cat
'''
select(sql)

,purpose_cat,count(*)
0,другое,3
1,машина,6
2,свадьба,5


In [ ]:
# дальше снова смотрим, что попало в другое - действуем итерациями
# используем подзапрос, где мы создаем рассчетные столбец purpose_cut

sql = f'''select t.purpose, count(*)
from ({cat}) t
where t.purpose_cat = 'другое'

group by t.purpose
order by count(*) desc'''

select(sql)

,purpose,count(*)
0,на возвращение 2007,1
1,на покупку недвижимости,1
2,недвижимость,1


In [ ]:
# Продолжаем так пока не иссякнет категория 'другое'